In [163]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 

driver = webdriver.Chrome()
url = "https://taas.koroad.or.kr/sta/acs/exs/typical.do?menuId=WEB_KMP_OVT_UAS_ASA"
driver.get(url)

In [164]:
# 좌측 교통사고 (경찰DB) 클릭 
driver.find_element(By.CSS_SELECTOR,"#jdMenuTree17").click()

In [165]:
# 사고일반
driver.find_element(By.CSS_SELECTOR,"#ddMenuTree17 > div:nth-child(5) > a.node").click()

In [166]:
import time 
# 사고유형별 시간대별 교통사고 클릭 
driver.find_element(By.CSS_SELECTOR,"#sdMenuTree32").click()


In [167]:
# 년도 2014년 시작
driver.find_element(By.CSS_SELECTOR,"#startYear > option:nth-child(10)").click() 

In [168]:
# 년도 2014년 끝
driver.find_element(By.CSS_SELECTOR,"#endYear > option:nth-child(10)").click()   

In [169]:
# 조회
driver.find_element(By.CSS_SELECTOR,"#searchDiv > ul.top01-03 > li:nth-child(2) > input").click()
#time.sleep(5)

In [171]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# iframe 요소 찾기
iframes = driver.find_elements(By.TAG_NAME,"iframe")
print("Number of iframes:", len(iframes))
print(iframes)
# iframe 으로 전환 
driver.switch_to.frame('eosFrame')

Number of iframes: 1
[<selenium.webdriver.remote.webelement.WebElement (session="b255c15e23538b09ee38b0d09546d07b", element="f.874173810D1619B0EA4715A07BAFA7B1.d.C9BFD6EC378790F9C76CD13491EAAB4E.e.5527")>]


NoSuchFrameException: Message: eosFrame


In [172]:
# 전환된 iframe 내 html
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup

<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=EmulateIE7" http-equiv="X-UA-Compatible"/>
<title>Typical Analysis</title>
<link href="../common/img/yain_weblogo.gif" rel="shortcut icon"/>
<link href="/OBIP/common/css/common.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/analysis/css/analysis.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/analysis/octagon/css/octagon.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/analysis/octagon/css/style.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/OctagonGridStyleCss.jsp?styleId=5" id="styleInf" rel="stylesheet" type="text/css"/>
<script src="/OBIP/common/js/prototype.js" type="text/javascript"></script>
<script src="/OBIP/common/js/jquery-1.9.1.js" type="text/javascript"></script>
<script src="/OBIP/common/js/jquery-ui.js" type="text/javascript"></script>
<script type="text/javascript">
jQuery.noConflict();
var j$ = jQuery;
</script>
<!--script type="t

In [161]:
import pandas as pd

# 1. table 접근 후 데이터 파싱 (사고유형 대/중 분류)
table_rows = soup.select("#OctagonGrid > tbody > tr")
table_info = []

# rowspan 때문에 대분류/중분류가 생략되는 셀이 생기므로, 이전 값 기억하도록
curr_topcat = ""
curr_midcat = ""

for row in table_rows:
    cells = row.find_all("td")
    texts = [cell.text.strip() for cell in cells]

    if not texts:
        continue

    if len(cells) == 3:
        curr_topcat = texts[0]
        curr_midcat = texts[1]
        구분 = texts[2]
    
    elif len(cells) == 2:
        curr_midcat = texts[0]
        구분 = texts[1]

    elif len(cells) == 1:
        구분 = texts[0]
    
    else:
        continue

    table_info.append((curr_topcat,curr_midcat,구분))


# 2. 실제 수치가 담긴 테이블에서 값 추출
data_rows = soup.select("#dataSession > tbody > tr")

# 3. 시간대 헤더는 고정이므로 수동 정의

시간대_headers = ['00~02시', '02~04시', '04~06시', '06~08시', '08~10시',
                 '10~12시', '12~14시', '14~16시', '16~18시', '18~20시',
                 '20~22시', '22~24시']

# 4. 병렬로 라벨 + 데이터 묶어서 정리 
data = []

for idx, row in enumerate(data_rows):
    cells = row.find_all("td")
    values = [cell.text.strip().replace(',','') for cell in cells]

    if idx >= len(table_info):
        continue

    대분류, 중분류, 구분 = table_info[idx]

    if len(values) != len(시간대_headers):
        print(f"값 개수 불일치 : {values}")
        continue

    for 시간대, 값 in zip(시간대_headers,values):
        data.append({
            "사고유형대분류": 대분류,
            "사고유형중분류": 중분류,
            "구분": 구분,
            "시간대":시간대,
            "값": 값
        })

# driver.quit()

# 데이터프레임 생성 
df = pd.DataFrame(data)
df.to_csv("시간대별_사고통계.csv",index=False, encoding='utf-8-sig')
print("저장완료")       

저장완료


In [173]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://taas.koroad.or.kr/sta/acs/exs/typical.do?menuId=WEB_KMP_OVT_UAS_ASA")

total_data = []

for year in range(2014, 2024):
    print(f"📆 {year}년 조회 중...")

    # 1. 연도 선택 드롭다운
    year_select = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "searchYear"))
    ))
    year_select.select_by_value(str(year))
    time.sleep(0.5)

    # 2. 조회 버튼 클릭
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "searchBtn"))
    ).click()
    time.sleep(1.5)  # 로딩 대기

    # 3. iframe 전환
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.NAME, "eosFrame"))
    )

    # 4. soup 생성
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 5. 사고유형 + 구분 라벨 추출
    label_rows = soup.select("#OctagonGrid > tbody > tr")
    label_info = []

    curr_topcat = ""
    curr_midcat = ""

    for row in label_rows:
        cells = row.find_all("td")
        texts = [cell.text.strip() for cell in cells]

        if not texts:
            continue

        if len(cells) == 3:
            curr_topcat = texts[0]
            curr_midcat = texts[1]
            구분 = texts[2]
        elif len(cells) == 2:
            curr_midcat = texts[0]
            구분 = texts[1]
        elif len(cells) == 1:
            구분 = texts[0]
        else:
            continue

        label_info.append((curr_topcat, curr_midcat, 구분))

    # 6. 값 테이블 추출
    data_rows = soup.select("#dataSession > tbody > tr")
    시간대_headers = ['00~02시', '02~04시', '04~06시', '06~08시', '08~10시',
                     '10~12시', '12~14시', '14~16시', '16~18시', '18~20시',
                     '20~22시', '22~24시']

    for idx, row in enumerate(data_rows):
        values = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
        if idx >= len(label_info):
            continue
        대분류, 중분류, 구분 = label_info[idx]

        if len(values) != len(시간대_headers):
            continue

        for 시간대, 값 in zip(시간대_headers, values):
            total_data.append({
                "연도": year,
                "사고유형대분류": 대분류,
                "사고유형중분류": 중분류,
                "구분": 구분,
                "시간대": 시간대,
                "값": 값
            })

    # 7. iframe 빠져나오기
    driver.switch_to.default_content()

driver.quit()

# 8. 저장
df = pd.DataFrame(total_data)
df.to_csv("2014_2023_시간대별_사고통계.csv", index=False, encoding='utf-8-sig')
print("✅ 전체 연도 데이터 저장 완료!")


📆 2014년 조회 중...


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF73B455335+78597]
	GetHandleVerifier [0x00007FF73B455390+78688]
	(No symbol) [0x00007FF73B2091AA]
	(No symbol) [0x00007FF73B25F149]
	(No symbol) [0x00007FF73B25F3FC]
	(No symbol) [0x00007FF73B2B2467]
	(No symbol) [0x00007FF73B28712F]
	(No symbol) [0x00007FF73B2AF2BB]
	(No symbol) [0x00007FF73B286EC3]
	(No symbol) [0x00007FF73B2503F8]
	(No symbol) [0x00007FF73B251163]
	GetHandleVerifier [0x00007FF73B6FEEED+2870973]
	GetHandleVerifier [0x00007FF73B6F9698+2848360]
	GetHandleVerifier [0x00007FF73B716973+2967875]
	GetHandleVerifier [0x00007FF73B47017A+188746]
	GetHandleVerifier [0x00007FF73B47845F+222255]
	GetHandleVerifier [0x00007FF73B45D2B4+111236]
	GetHandleVerifier [0x00007FF73B45D462+111666]
	GetHandleVerifier [0x00007FF73B443589+5465]
	BaseThreadInitThunk [0x00007FFF2D68259D+29]
	RtlUserThreadStart [0x00007FFF2EB2AF38+40]
